# Tips

## Differentiable smooth functions

Smooth and differentiable functions are more friendly for automatic differentiation (AD) and differential equation solvers.

### Smooth Heaviside step function

A [Heaviside step function](https://en.wikipedia.org/wiki/Heaviside_step_function) (0 when x < a, 1 when x > a) could be approximated with a steep [logistic function](https://en.wikipedia.org/wiki/Logistic_function).

In [ ]:
using Plots
Plots.default(fmt=:png)

In [ ]:
function smoothheaviside(x, k=100)
    1 / (1 + exp(-k * x))
end

The function output switches from zero to one around `x=0`.

In [ ]:
plot(smoothheaviside, -1, 1)

### Smooth single pulse

A single pulse could be approximated with a product of two logistic functions.

In [ ]:
function singlepulse(x, t0=0, t1=0.1, k=1000)
    smoothheaviside(x - t0, k) * smoothheaviside(t1 - x, k)
end

In [ ]:
plot(singlepulse, -1, 1)

### Smooth absolute value

Inspired by: https://discourse.julialang.org/t/smooth-approximation-to-max-0-x/109383/13

In [ ]:
# Approximate abs(x)
function smooth_abs(x; c=(1//2)^10)
    sqrt(x^2 + c^2) - c
end

In [ ]:
plot(smooth_abs, -10, 10, label="Smooth abs")

### Smooth max function

Approximate `max(0, x)`

In [ ]:
function smooth_max(x; c=(1//2)^10)
    0.5 * (x + smooth_abs(x;c))
end

In [ ]:
plot(smooth_max, -10, 10, label="Smooth max")

### Smooth minimal function

Approximate `min(0, x)`

In [ ]:
function smooth_min(x; c=(1//2)^10)
    0.5 * (smooth_abs(x;c) - x)
end

In [ ]:
plot(smooth_min, -10, 10, label="Smooth min")

### (Another) smooth step function

In [ ]:
function smooth_step(x; c=(1//2)^10)
    0.5 * (x / (sqrt(x^2 + c)) + 1)
end

In [ ]:
plot(smooth_step, -10, 10, label="Smooth step")

### Periodic pulses

From: https://www.noamross.net/2015/11/12/a-smooth-differentiable-pulse-function/

In [ ]:
function smoothpulses(t, tstart, tend, period=1, amplitude=period / (tend - tstart), steepness=1000)
    @assert tstart < tend < period
    xi = 3 / 4 - (tend - tstart) / (2 * period)
    p = inv(1 + exp(steepness * (sinpi(2 * ((t - tstart) / period + xi)) - sinpi(2 * xi))))
    return amplitude * p
end

In [ ]:
plot(t->smoothpulses(t, 0.2, 0.3, 0.5), 0.0, 2.0, lab="pulses")

## Avoid DomainErrors

Some functions such as `sqrt(x)`, `log(x)`, and `pow(x)`, throw `DomainError` exceptions with negative `x`, interrupting differential equation solvers. One can use the respective functions in https://github.com/JuliaMath/NaNMath.jl, returning `NaN` instead of throwing a `DomainError`. Then, the differential equation solvers will reject the solution and retry with a smaller time step.

In [ ]:
import NaNMath as nm
nm.sqrt(-1.0) ## returns NaN

## ODE function from an MTK ODE system

`f = ODEFunction(sys)` could be useful in visualizing vector fields.

In [ ]:
using ModelingToolkit
using DifferentialEquations
using Plots

Independent (time) and dependent (state) variables (x and RHS)

In [ ]:
@variables t x(t) RHS(t)

Setting parameters in the modeling

In [ ]:
@parameters τ

Differential operator w.r.t. time

In [ ]:
D = Differential(t)

Equations in MTK use the tilde character (`~`) as equality.
very MTK system requires a name. The `@named` macro simply ensures that the symbolic name matches the name in the REPL.

In [ ]:
eqs = [
    RHS  ~ (1 - x)/τ,
    D(x) ~ RHS
]

@named fol_separate = ODESystem(eqs, t)

In [ ]:
sys = structural_simplify(fol_separate)

In [ ]:
f = ODEFunction(sys)

In [ ]:
f([0.0], [1.0], 0.0) # f(u, p, t) returns the value of derivatives